# **CATEGORÍA FyV predictions**

## **Importar librerías**

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
import holidays

In [3]:
# Facebook Prophet
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

In [4]:
# evaluar error predicciones
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [5]:
# generar estructura de tablas y datos para exportar
from sqlalchemy import create_engine, MetaData, Column, Table, Integer, String, Float, Date, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

## **Cargar datos**

In [6]:
# dataframe generado en Workbench con productos de categoría "Fruta y verdura", con venta diaria histórica
prod_fecha_fyv = pd.read_csv('../../data/sql/venta/prod_fecha_fyv.csv')

# dataframe "ventas_productos", con datos de venta diaria histórica
ventas_productos = pd.read_csv('../../data/ventas_productos.csv')

In [7]:
prod_fecha_fyv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26425 entries, 0 to 26424
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   fecha       26425 non-null  object
 1   dia_semana  26425 non-null  object
 2   categoria   26425 non-null  object
 3   producto    26425 non-null  object
 4   formato     26425 non-null  object
 5   venta       26425 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 1.2+ MB


In [8]:
ventas_productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26425 entries, 0 to 26424
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   venta_id     26425 non-null  int64
 1   producto_id  26425 non-null  int64
 2   venta        26425 non-null  int64
dtypes: int64(3)
memory usage: 619.5 KB


## **Transformar datos**

In [9]:
# cambiar columna "fecha" sea tipo "datetime"
prod_fecha_fyv['fecha'] = pd.to_datetime(prod_fecha_fyv['fecha'])

prod_fecha_fyv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26425 entries, 0 to 26424
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   fecha       26425 non-null  datetime64[ns]
 1   dia_semana  26425 non-null  object        
 2   categoria   26425 non-null  object        
 3   producto    26425 non-null  object        
 4   formato     26425 non-null  object        
 5   venta       26425 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 1.2+ MB


In [10]:
# definir rango de fechas de predicción
fecha_inicio = pd.to_datetime('2023-06-01')
fecha_fin = pd.to_datetime('2023-06-30')

# eliminar filas dentro de rango de fechas para generar predicciones futuras
prod_fecha_fyv = prod_fecha_fyv[~((prod_fecha_fyv['fecha'] >= fecha_inicio) & (prod_fecha_fyv['fecha'] <= fecha_fin))]

prod_fecha_fyv

,fecha,dia_semana,categoria,producto,formato,venta
0,2023-01-02,lunes,Fruta y verdura,Plátano de Canarias,Pieza 160 g aprox.,4
1,2023-01-02,lunes,Fruta y verdura,Banana,Pieza 170 g aprox.,14
2,2023-01-02,lunes,Fruta y verdura,Plátano macho,Pieza 340 g aprox.,26
3,2023-01-02,lunes,Fruta y verdura,Uva blanca sin semillas,Bandeja 500 g aprox.,36
4,2023-01-02,lunes,Fruta y verdura,Uva roja sin semillas,Bandeja 500 g aprox.,2
...,...,...,...,...,...,...
21870,2023-05-31,miércoles,Fruta y verdura,Mix de verduras paisana para micro,Paquete 300 g,25
21871,2023-05-31,miércoles,Fruta y verdura,Sopa juliana para micro,Paquete 400 g,19
21872,2023-05-31,miércoles,Fruta y verdura,Patata para micro,Paquete 400 g,22
21873,2023-05-31,miércoles,Fruta y verdura,Batata para micro,Paquete 400 g,3


In [11]:
# generar nueva columna con nombre único para identificar cada combinación de producto y formato
prod_fecha_fyv['producto_formato'] = prod_fecha_fyv['producto'] + '_' + prod_fecha_fyv['formato']

# obtener lista única de nombres de producto en "prod_fecha_fyv"
nombres_productos = prod_fecha_fyv['producto_formato'].unique()

# generar diccionario para almacenar subdataframes separados por "producto_formato"
dfs_productos = {}

# recorrer lista "nombres_productos" y filtrar "prod_fecha_fyv" por cada nombre
for producto_formato in nombres_productos:
    df_producto = prod_fecha_fyv[prod_fecha_fyv['producto_formato'] == producto_formato].copy()

     # reiniciar índice subdataframe
    df_producto.reset_index(drop=True, inplace=True)

    dfs_productos[producto_formato] = df_producto

## **Predicciones**

### **Prophet**

In [12]:
# realizar función para generar predicciones de venta futuras para cada producto con Prophet
def preds_prophet(subdf_producto):
    # crear objeto de Prophet
    m = Prophet()

    # renombrar columnas requeridas para aplicar Prophet ("ds" para columna "fecha", "y" para columna de "venta")
    df_prophet = subdf_producto[['fecha', 'venta']].rename(columns={'fecha' : 'ds', 'venta' : 'y'})

    # ajustar modelo Prophet
    m.fit(df_prophet)

    # generar predicciones para 30 días futuros a los datos históricos dados
    futuras_fechas = m.make_future_dataframe(periods=30)
    predicciones_prophet = m.predict(futuras_fechas)

    return predicciones_prophet

In [13]:
# generar diccionario para almacenar subdataframes con predicciones separados por "producto_formato"
preds_productos = {}

# recorrer subdataframes generados y aplicar modelo Prophet en cada uno para generar predicciones
for producto_formato, subdf_producto in dfs_productos.items():
    predicciones_prophet = preds_prophet(subdf_producto)
    preds_productos[producto_formato] = predicciones_prophet

17:04:21 - cmdstanpy - INFO - Chain [1] start processing
17:04:21 - cmdstanpy - INFO - Chain [1] done processing
17:04:21 - cmdstanpy - INFO - Chain [1] start processing
17:04:21 - cmdstanpy - INFO - Chain [1] done processing
17:04:21 - cmdstanpy - INFO - Chain [1] start processing
17:04:21 - cmdstanpy - INFO - Chain [1] done processing
17:04:21 - cmdstanpy - INFO - Chain [1] start processing
17:04:21 - cmdstanpy - INFO - Chain [1] done processing
17:04:21 - cmdstanpy - INFO - Chain [1] start processing
17:04:21 - cmdstanpy - INFO - Chain [1] done processing
17:04:22 - cmdstanpy - INFO - Chain [1] start processing
17:04:22 - cmdstanpy - INFO - Chain [1] done processing
17:04:22 - cmdstanpy - INFO - Chain [1] start processing
17:04:22 - cmdstanpy - INFO - Chain [1] done processing
17:04:22 - cmdstanpy - INFO - Chain [1] start processing
17:04:22 - cmdstanpy - INFO - Chain [1] done processing
17:04:22 - cmdstanpy - INFO - Chain [1] start processing
17:04:22 - cmdstanpy - INFO - Chain [1]

In [14]:
# generar diccionario para almacenar subdataframes con columnas de "fecha" y "prevision"
cols_objetivo = ['ds', 'yhat']
preds_prophet_final = {}

# recorrer subdataframes con predicciones generadas por Prophet, y seleccionar columnas "ds" como "fecha" y "yhat" como "prevision"
for producto_formato, predicciones in preds_productos.items():
    predicciones_seleccionadas = predicciones[cols_objetivo].copy()
    predicciones_seleccionadas = predicciones_seleccionadas.rename(columns={'ds' : 'fecha', 'yhat' : 'prevision'})
    predicciones_seleccionadas['prevision'] = predicciones_seleccionadas['prevision'].astype(int)
    preds_prophet_final[producto_formato] = predicciones_seleccionadas

In [15]:
# unir subdataframes en un dataframe global
preds_prophet_fyv = pd.concat(list(preds_prophet_final.values()), ignore_index=True)

preds_prophet_fyv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27125 entries, 0 to 27124
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   fecha      27125 non-null  datetime64[ns]
 1   prevision  27125 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 424.0 KB


In [16]:
# ajustar dataframe global al formato de la tabla "ventas_productos"

# obtener festivos nacionales en España para el año 2023
festivos_espana = holidays.Spain(years=2023)

# eliminar filas que sean domingos en la columna "fecha"
preds_prophet_fyv = preds_prophet_fyv[~(preds_prophet_fyv['fecha'].dt.dayofweek == 6)]

# eliminar filas correspondientes a festivos nacionales en España
preds_prophet_fyv = preds_prophet_fyv[~preds_prophet_fyv['fecha'].isin(festivos_espana)]

# reiniciar índice del dataframe
preds_prophet_fyv.reset_index(drop=True, inplace=True)

preds_prophet_fyv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26425 entries, 0 to 26424
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   fecha      26425 non-null  datetime64[ns]
 1   prevision  26425 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 413.0 KB


In [18]:
# medir error predicciones con "Error Medio Absouluto" (MAE)

# datos reales y predicciones
y_real_prophet = ventas_productos['venta']
y_pred_prophet = preds_prophet_fyv['prevision']

# calcular MAE
mae_prophet = mean_absolute_error(y_real_prophet, y_pred_prophet)

# resultado
print('MAE de Prophet: ', mae_prophet)

MAE de Prophet:  11.636480605487227


In [19]:
# medir error predicciones con "Error Cuadrático Medio" (MSE) y "Raíz del Error Cuadrático Medio" (RMSE)

# calcular MSE
mse_prophet = mean_squared_error(y_real_prophet, y_pred_prophet)

# calcular RMSE
rmse_prophet = mean_squared_error(y_real_prophet, y_pred_prophet, squared=False)

# resultados
print('MSE de Prophet: ', mse_prophet)
print('RMSE de Prophet: ', rmse_prophet)

MSE de Prophet:  194.6412866603595
RMSE de Prophet:  13.951390133616059


In [20]:
# añadir predicciones a columna "prevision"
ventas_productos['prevision'] = preds_prophet_fyv['prevision']

ventas_productos

,venta_id,producto_id,venta,prevision
0,1,1,4,21
1,2,1,28,14
2,3,1,3,19
3,4,1,12,14
4,5,1,13,26
...,...,...,...,...
26420,147,175,0,23
26421,148,175,0,14
26422,149,175,0,21
26423,150,175,0,17


## **Exportar predicciones**

In [27]:
# exportar predicciones transformadas a ".csv"
ventas_productos.to_csv('../../data/ventas_productos.csv', index=False)